In [3]:
import pandas as pd
import numpy as np

In [4]:
plays = pd.read_csv('dataset/user_artists.dat', sep='\t')
artists = pd.read_csv('dataset/artists.dat', sep='\t', usecols=['id','name'])

# Merge artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [5]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)

# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)
#print(ap)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values

# Show sparsity
sparsity = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("sparsity: %.2f" % sparsity)

sparsity: 0.28


In [6]:
from scipy.sparse import csr_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


In [54]:
### from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
print("X:", X.shape)
Xcoo = X.tocoo()
print("Xcoo:", Xcoo.shape)

data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

X: (1892, 17632)
Xcoo: (1892, 17632)


* ### WARP (Weighted Approximate-Rank Pairwise)

In [8]:
# Train
model = LightFM(learning_rate=0.05, loss='warp')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [10]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.39, test 0.14.
AUC: train 0.97, test 0.86.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [14]:
# Predict
scores_warp = model.predict(0, np.arange(n_items))
top_items_warp = artist_names[np.argsort(-scores_warp)]
print(top_items_warp)

['Depeche Mode' 'Pet Shop Boys' 'Duran Duran' ... 'Avril Lavigne'
 'Miley Cyrus' 'Rihanna']


* ### BPR (Bayesian Personalised Ranking)

In [15]:
# Train
model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [16]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.36, test 0.12.
AUC: train 0.84, test 0.78.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [21]:
# Predict
scores_bpr = model.predict(0, np.arange(n_items))
top_items_bpr = artist_names[np.argsort(-scores_bpr)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)


Top items BPR: 
 ['Lady Gaga' 'Britney Spears' 'The Beatles' ... 'Planningtorock'
 'So Long Forgotten' 'Herman Düne']
Top items WARP: 
 ['Depeche Mode' 'Pet Shop Boys' 'Duran Duran' ... 'Avril Lavigne'
 'Miley Cyrus' 'Rihanna']


* ### Logistic

In [22]:
# Train
model = LightFM(learning_rate=0.05, loss='logistic')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [23]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.20, test 0.07.
AUC: train 0.89, test 0.81.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [24]:
# Predict
scores_log = model.predict(0, np.arange(n_items))
top_items_log = artist_names[np.argsort(-scores_log)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)
print("Top items Logistic: \n", top_items_log)

Top items BPR: 
 ['Lady Gaga' 'Britney Spears' 'The Beatles' ... 'Planningtorock'
 'So Long Forgotten' 'Herman Düne']
Top items WARP: 
 ['Depeche Mode' 'Pet Shop Boys' 'Duran Duran' ... 'Avril Lavigne'
 'Miley Cyrus' 'Rihanna']
Top items Logistic: 
 ['Lady Gaga' 'Britney Spears' 'Rihanna' ... 'The Lively Ones'
 'Dark Lunacy' 'Bride']


In [68]:
# Predict
scores_bpr = model.predict(0, np.arange(n_items))
top_items_bpr = artist_names[np.argsort(-scores_bpr)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)


Top items BPR: 
 ['Van Halen' 'The Smiths' 'Siouxsie and the Banshees' ... 'Britney Spears'
 'Rihanna' 'Miley Cyrus']
Top items WARP: 
 ['Depeche Mode' 'Duran Duran' 'The Beatles' ... 'Common Rotation'
 'Dissimulation' 'Aidonia']


In [70]:
# Train
model = LightFM(learning_rate=0.05, loss='logistic')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [71]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.20, test 0.07.
AUC: train 0.89, test 0.81.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [72]:
# Predict
scores_log = model.predict(0, np.arange(n_items))
top_items_log = artist_names[np.argsort(-scores_log)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)
print("Top items Logistic: \n", top_items_log)


Top items BPR: 
 ['Van Halen' 'The Smiths' 'Siouxsie and the Banshees' ... 'Britney Spears'
 'Rihanna' 'Miley Cyrus']
Top items WARP: 
 ['Depeche Mode' 'Duran Duran' 'The Beatles' ... 'Common Rotation'
 'Dissimulation' 'Aidonia']
Top items Logistic: 
 ['Lady Gaga' 'Britney Spears' 'Rihanna' ... 'PlatEAU' 'Deacon Blue'
 'Преслава']


* ### K-OS WARP

In [116]:
# Train
model = LightFM(learning_rate=0.05, loss='warp-kos')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [117]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.35, test 0.12.
AUC: train 0.89, test 0.82.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [ ]:
# Predict
scores_warp-kos = model.predict(0, np.arange(n_items))
top_items_warp-kos = artist_names[np.argsort(-scores_warp-kos)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)
print("Top items Logistic: \n", top_items_log)
print("Top items k-OS Warp: \n", top_items_warp-kos)

In [68]:
# Predict
scores_bpr = model.predict(0, np.arange(n_items))
top_items_bpr = artist_names[np.argsort(-scores_bpr)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)


Top items BPR: 
 ['Van Halen' 'The Smiths' 'Siouxsie and the Banshees' ... 'Britney Spears'
 'Rihanna' 'Miley Cyrus']
Top items WARP: 
 ['Depeche Mode' 'Duran Duran' 'The Beatles' ... 'Common Rotation'
 'Dissimulation' 'Aidonia']


In [70]:
# Train
model = LightFM(learning_rate=0.05, loss='logistic')
model.fit(train, epochs=10, num_threads=2)

Measure the precision at k metric for a model: the fraction of known positives in the first k positions of the ranked list of results. A perfect score is 1.0.

In [71]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

# Measure the ROC AUC metric for a model: the probability that a randomly chosen positive 
# example has a higher score than a randomly chosen negative example. 
# A perfect score is 1.0.

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.20, test 0.07.
AUC: train 0.89, test 0.81.


Measure the recall at k metric for a model: the number of positive items in the first k positions of the ranked list of results divided by the number of positive items in the test period. A perfect score is 1.0.

Measure the reciprocal rank metric for a model: 1 / the rank of the highest ranked positive example. A perfect score is 1.0.

In [72]:
# Predict
scores_log = model.predict(0, np.arange(n_items))
top_items_log = artist_names[np.argsort(-scores_log)]
print("Top items BPR: \n", top_items_bpr)
print("Top items WARP: \n", top_items_warp)
print("Top items Logistic: \n", top_items_log)


Top items BPR: 
 ['Van Halen' 'The Smiths' 'Siouxsie and the Banshees' ... 'Britney Spears'
 'Rihanna' 'Miley Cyrus']
Top items WARP: 
 ['Depeche Mode' 'Duran Duran' 'The Beatles' ... 'Common Rotation'
 'Dissimulation' 'Aidonia']
Top items Logistic: 
 ['Lady Gaga' 'Britney Spears' 'Rihanna' ... 'PlatEAU' 'Deacon Blue'
 'Преслава']


In [64]:
def scoring():
    
    learning_rate = [0.03, 0.05, 0.08, 0.10, 0.12, 0.14]
    losslist = ['logistic', 'bpr', 'warp', 'warp-kos']
    klist = [3, 5, 7, 9, 11, 13]
    results = []
    
    for x in learning_rate:
        for y in losslist:
            for z in klist:
            
                model = LightFM(learning_rate=x, loss = y)
                model.fit(train, epochs=10, num_threads=2)

                trainPrecision = precision_at_k(model, train, k=z).mean()
                testPrecision = precision_at_k(model, test, k=z, train_interactions=train).mean()

                trainAUC = auc_score(model, train).mean()
                testAUC = auc_score(model, test, train_interactions=train).mean()

                dicttemp = {}
                dicttemp = {'K':z, 'Name':y, 'Learning Rate':x, 'Train Precision':trainPrecision, 'Train AUC':trainAUC, 'Test Precision':testPrecision, "Train AUC":trainAUC, "Test AUC":testAUC}

                results.append(dicttemp)
            
    results = pd.DataFrame(results)
    
    return results

In [65]:
scoring()

,K,Name,Learning Rate,Train Precision,Train AUC,Test Precision,Test AUC
0,3,logistic,0.03,0.229421,0.886406,0.089244,0.807125
1,5,logistic,0.03,0.217313,0.886890,0.080747,0.806915
2,7,logistic,0.03,0.207799,0.887001,0.073752,0.807343
3,9,logistic,0.03,0.199622,0.886737,0.069748,0.807469
4,11,logistic,0.03,0.195095,0.886657,0.068800,0.807637
...,...,...,...,...,...,...,...
139,5,warp-kos,0.14,0.277005,0.878168,0.095147,0.794324
140,7,warp-kos,0.14,0.300281,0.878412,0.100571,0.793915
141,9,warp-kos,0.14,0.296926,0.879012,0.094519,0.794209
142,11,warp-kos,0.14,0.288756,0.879915,0.088533,0.798933


## The best "Test AUC" score (85,70%) is done with a "learning rate": 5% and "k"=7

In [ ]:
# Best params with GridSearchCV

from sklearn.model_selection import GridSearchCV


In [ ]:
A continuer